This notebook is part of the [Machine Learning class](https://github.com/erachelson/MLclass) by [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en).

This notebook is joint work by Erwan Lecarpentier, Luca Mossina and Emmanuel Rachelson.

License: CC-BY-SA-NC.

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Support Vector Machines (practice session)</div>

# Introduction

SVMs is a classical Supervised Learning technique.

Content:
1. [Linear SVMs](#sec1)<br>
1.1 [Let's play](#subsec11)<br>
1.2 [Tune your linear SVM](#subsec12)
2. [SVM with kernels](#sec2)<br>
2.1 [Warm-up: Americans and Atheism  (OPTIONAL)](#subsec21)<br>
2.2 [Back to basics](#subsec22)<br>
2.3 [Kernels](#subsec23)
3. [Application](#sec3)

# <a id="sec1"></a> 1. Linear SVMs

The goal of this section is to explore the most basic form of SVM, namely the linear one. We are going to test this method on a toy data set and to play with the hyperparameters in order to get the best classifier.

<div class="alert alert-warning">**Question 1.1**<br>
What is the objective of SVMs?<br>
On which principle is it built? Give an **intuitive** geometrical interpretation.
</div>

Your answer :

## <a id="subsec11"></a> 1.1 Let's play

Let's take a toy data set $\{ {x_i, y_i} \}_{i=1}^n$ where $x_i \in \mathbb{R}^2$ is a point and $y_i \in [-1, 1]$ a class.<br>
The code below allows you to load the data and to display it.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
fig_size = (6,6)

csv_file_name="data/data1.csv"
res = np.loadtxt(csv_file_name, delimiter=',')
X = res[:,0:-1]
y = res[:,-1].astype(int)
Xblue = X[y==-1]
Xred  = X[y==+1]
fig = plt.figure(figsize=fig_size, dpi=80, facecolor='w', edgecolor='k')
plt.scatter(Xblue[:,0], Xblue[:,1], c='c', s=20)
plt.scatter(Xred[:, 0], Xred[:, 1], c='r', s=20);

<div class="alert alert-warning">**Question 1.1.1**<br>
Can this data set be completely separated by a straight line?<br>
What kind of technique could be used to train a classifier on this data set?
</div>

Your answer:

Let's apply the SVM method to this dataset. To this purpose, we are going to use scikit-learn, a Python library providing most of the common Machine Learning off the shelf methods including SVM.  
Their documentation page for this method can be found here: http://scikit-learn.org/stable/modules/svm.html

<div class="alert alert-warning">**Question 1.1.2**<br>
Use the class sklearn.svm.SVC provided in scikit-learn in order to train an SVM on the provided data.
Try to display the separating hyperplane on this dataset. You can get inspiration from the SVM course's code.
</div>

In [ ]:
# %load solutions/code1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).



<div class="alert alert-warning">**Question 1.1.3**<br>
In "Support Vector Machine", what are the "support vectors"?
</div>

Your answer :

## <a id="subsec12"></a> 1.2 Tune your linear SVM

One of the parameters of the class sklearn.svm.SVC is the penalty parameter $C$ of the error term. It quantifies the missmatch-tolerance that is allowed when fitting an SVM model to the data. The smaller, the bigger is the tolerance. In our case, having such a tolerance is necessary because the data cannot be separated by a straight line, thus some point have to be on the wrong side of the separating hyperplan.

$C$ is an hyperparameter, which means that it needs to be tuned for the algorithm to yield good results. This is a common concern in Machine Learning, as a result, techniques have been designed in order to select the best hyperparameters. The most common technique is probably the cross validation. Roughly it consists in:

    0) dividing the data set into training and test sets;

    1) selecting a value of the hyperparameters;

    2) train the model with the training set;

    3) test the model with the test set and compute a performance indicator;

    4) go back to 1) with another value of the hyperparameters.

When the procedure is finished, simply choose the hyperparameters yielding the best performance. We are now going to perform k-fold cross validation - which is a certain type of cross validation - on our problem, in order to identify the best value of $C$.
For more details, see the wikipedia article: https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation

For more details on the type of indicators used in step 3), see https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers<br>
The accuracy, refered to as ACC in the article, may be a good start (and most scikit-learn classifiers implement it directly via the [`score` function](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score)).

<div class="alert alert-warning">**Question 1.2.1**<br>
Fill the code below in order to find the best value of the hyperparameter $C$.
</div>

In [ ]:
C = [0.001, 0.01, 0.1, 1.0, 10, 25, 30, 42, 100, 1000] # Tested values of C
k = 10 # k-fold CV: number of subsets
n = int(len(X)/k) # length of subsets

In [ ]:
# %load solutions/code2.py
### WRITE YOUR CODE HERE - implement your own version of k-fold cross-validation.
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning">**Question 1.2.2**<br>
What is the best value of $C$ among the proposed ones?<br>
What can you expect from the curve of the accuracy as a function of $C$ if you used a finer meshing for $C$?
</div>

Your answer :

# <a id="sec2"></a> 2. SVMs with kernels

In [ ]:
# === imports
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import copy

%load_ext autoreload
%autoreload 2

## Binary Classification

The natural application of SVM methods is that of binary classification.

## <a id="subsec21"></a> 2.1 Warm-up: Americans and Atheism  (OPTIONAL)

**SCOPE**: warm-up, revise and challenge yourself with a simple example.
    - MAXIMUM TIME: 10 minutes, afterwards look at the proposed solution.

Let us find out how americans tolerated atheists in 1976.
The dataset is described as follows:

```
Dataset:  atheist.dat
Source: E. Filsinger (1976). "Tolerance of Non-Believers: A Cross-Tabular  and Log Linear Analysis of Some Religious Correlates," Review of Religious  Research, Vol.17, #3, pp.232-240
Description: Church Attendance and Tolerance for Atheists for survey of 1221 people.

Variables/Columns 
Church Attendance  8  /* 1=Never, 2=Yearly, 3=Monthly, 4=Weekly */
Tolerance for Atheists  16 /* 1=Low, 2=High  */
```

### 2.1.1 Retrieve and understand data

The file is available online at: http://users.stat.ufl.edu/~winner/datasets.html

<div class="alert alert-warning">**Question 2.1.1.1**<br>
Import the data as a pandas dataframe. No cheating, it can be done with a pandas function (one line).
Your dataframe should have 1221 rows and 2 columns, which you'll name respectively 'attendance', 'tolerance'.
<br><br>
Plot the data. 
</div>

In [ ]:
# %load solutions/code3.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


It doesn't look like the fancy figures we saw in class, does it?

### 2.1.2 Run SVM. Does it make sense?

<div class="alert alert-warning">**Question 2.1.2.1**<br>
First of all, what can we learn having a strong look at the data and their description?
<br>
Do we have categorical, binary, continuous data?
</div>

Your answer :

These are fundamental questions, which you'll ask yourselves for the rest of your (data science) life.
Often, collegues or clients have no idea about these formalities; they have a problem and they ask you to provide a good enough solution.

Let the fun begin. We want to do SVM classification with scikit-learn, but we've already forgotten its api!  
Thou shalt fear no more, the word of the Doc is with us:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm

We are doing SVM classification, could it be: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC ?
Let us copy-paste the example at the bottom of the page. Run the following:

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X, y)

Wait, what are `X` and `y`? 
Let us take a moment to ponder on how to name things.  
For sure, "x" and "y" mean something to all those with scientific background.  
Often, however, when we do some ML modeling, we are having a data-dialogue with someone, ourselves included.  
Do the favor to the "you" of the future and use a meaningful description for the variables; code is mostly meant to be read.

In [ ]:
attend = df['attendance'] # WARNING: you are passing a reference.
toler  = df['tolerance']

clf = SVC(gamma='auto')
clf.fit(X = attend,
        y = toler) 

Something just went horribly wrong.  
He's telling us something about ```"Expected 2D array, got 1D array instead: [...]"```  
After some intense googling, stackoverflow.com gives us a hint:  
https://stackoverflow.com/questions/38657138/scikits-learn-svm-1-dimensional-separating-hyperplane

Let us add a dummy column of zeros:

In [ ]:
df['zeros'] = 0 # dummy zeros

df  = df.sample(frac=1) # shuffle rows to avoid introducing bias
print(df.iloc[0:5,])

NTRAINS = 800 # out of 1221

tr = df.iloc[0:NTRAINS,:] # training data
te = df.iloc[NTRAINS: ,:] # testing data

# assert: make sure now rows were lost or doubled
assert tr.shape[0] + te.shape[0] == df.shape[0]

features = ['attendance', 'zeros']
attend = df[features]
toler  = df['tolerance']

from sklearn.svm import LinearSVC
clf = LinearSVC(tol=1e-5) # linear
# __uncomment__ if you want to try with nonlinear SVM:
# clf = SVC(gamma='auto')

clf.fit(X=attend[0:NTRAINS], y=toler[0:NTRAINS]) 

y_pred = clf.predict(attend[NTRAINS:])
y_test = toler[NTRAINS:].tolist()

errors = np.zeros(len(y_pred))

for i in range(len(errors)):
    if y_pred[i] != y_test[i]:
        errors[i] = 1
        
er_rate = np.sum(errors) / len(errors)
print("error rate:", er_rate)
print(" --> about ", int(er_rate*100), "% of the predictions were not correct.")

*Campeones*, we made it, sklearn is no more mad at us.  
Did we get anything interesting? Are we learning something at all?

**--> Lesson learned: some data are more svm-able than others**

Although SVM's are among the best tools in machine learning, we cannot expect it to work great in all cases.  
Much more could be done, tuning parameters or changing kernels, but let's go on to more interesting data

#### Bonus: beware of the Python   
Let's look at what done just before:

In [ ]:
# execute and ignore the following two lines
import copy            # IGNORE
dF = copy.deepcopy(df) # IGNORE

mangueira = dF['attendance'] # WARNING: you are passing a reference.
portela   = dF['tolerance']

# See how dangerous things can get:
print(mangueira[0:2])
mangueira[0:2] = 666, 666
print(dF.iloc[0:2]) # original df is modified!!

# Possible solution:
import copy
mangueira = copy.deepcopy(df['attendance'])
# all data is hard copied in new struct: can be a problem, if data is big

Let us move to more serious matters.

## <a id="subsec22"></a> 2.2 Back to basics (OPTIONAL)

After our previous little experiments, it is time to start gaining some insight into the use of kernels.  
But what _is_ a kernel? Can we get an intuitive image of what these kernels are doing?

In [ ]:
NPOINTS = 100
COLORS  = ['pink', "green"]

# Generate toy data
x = np.random.rand(NPOINTS)
y = np.ones(NPOINTS, dtype=int)
for i in range(NPOINTS):
    if x[i] < 0.0666 or x[i] > 0.666:
        y[i] = -1
        
# Plot toy data: we have a one-dimensional problem
plt.scatter(x[y==-1], np.zeros(len(x[y==-1])), color=COLORS[0])
plt.scatter(x[y== 1], np.zeros(len(x[y==1])), color=COLORS[1])
plt.xlabel('x');plt.ylabel('y')
plt.show()

A simple linear separator, a point in this example, cannot be found to separate the data.  
However we can easily see that the limit between the pink and the green regions are well defined.  
How can we **transform** the data to make this linear separation possible? We can apply a function $f : x_i \mapsto f(x_i)$ on each data point. Such function is called **feature** of the data.  
Let us try with a polynomial.

In [ ]:
a, b, c = 1, 1, 1
feature = a*x**2 + b*x + c

plt.scatter(x[y==-1], feature[y==-1], color=COLORS[0])
plt.scatter(x[y== 1], feature[y== 1], color=COLORS[1])
plt.xlabel('x')
plt.ylabel('f(x)')

In [ ]:
a, b, c = -8, 1, 2
feature = a*x**2 + b*x + c

plt.scatter(x[y==-1], feature[y==-1], color=COLORS[0])
plt.scatter(x[y== 1], feature[y== 1], color=COLORS[1])
plt.xlabel('x')
plt.ylabel('f(x)')

<div class="alert alert-warning">**Question 2.2.1**<br>
Copy the code above and play manually with the parameters in order to make the features separable.
</div>

In [ ]:
a, b, c = ?,?,?
feature = a*x**2 + b*x + c

colors = ['pink', "green"]

plt.scatter(x[y==-1], feature[y==-1], color=COLORS[0])
plt.scatter(x[y== 1], feature[y== 1], color=COLORS[1])
plt.xlabel('x')
plt.ylabel('f(x)')

height = 0.90
plt.plot([0,1], [height, height], color='red')
height = 0.95
plt.plot([0,1], [height, height], color='blue')
height = 1.00
plt.plot([0,1], [height, height], color='orange')
plt.show()

Wouldn't be nice to have an algorithm doing that for us?

## <a id="subsec23"></a> 2.3 Kernels

Now let's play with the kernels in scikit-learn. As before, we are going to use the sklearn.svm.SVC class in order to fit SVMs to some data models. The 'kernel' attribute of the class allows you to use different kernels for the used models.

Below are displayed several data sets.

In [ ]:
def plot_data(csv_file_name):
    fig_size = (6, 6)
    res = np.loadtxt(csv_file_name, delimiter=',')
    X = res[:,0:-1]
    y = res[:,-1].astype(int)
    Xblue = X[y==-1]
    Xred  = X[y==+1]
    fig = plt.figure(figsize=fig_size, dpi=80, facecolor='w', edgecolor='k')
    plt.scatter(Xblue[:,0], Xblue[:,1], c='c', s=20)
    plt.scatter(Xred[:, 0], Xred[:, 1], c='r', s=20)

plot_data("data/data2.csv")
plot_data("data/data3.csv")
plot_data("data/data4.csv")

<div class="alert alert-warning">**Question 2.3.1**<br>
Find a good kernel for each one of the data set above.<br><br>
**Hint 1**: you can reuse the `plot_SVC` function provided in class that plots the decision frontier of the model and the data set $(X, y)$.<br><br>
**Hint 2**: sometimes, the default kernels provided by scikit-learn are not enough. In this case, you can design a special kernel for your problem using the a callable function. See [here](http://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html) for an example. Recall that $k(x,x')$ measures how close $x$ and $x'$ are in the kernel's projection space: keeping this in mind can help you shape a good projection first, and then deriving a good kernel.
</div>

In [ ]:
from myplot import plot_SVC

def load_data(filename):
    res = np.loadtxt(filename, delimiter=',')
    X = res[:,0:-1]
    y = res[:,-1].astype(int)
    return X, y

In [ ]:
# %load solutions/code4.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning">**Question 2.3.2**<br>
Can you hint a method for robustly selecting the best kernel and or the hyperparameters of the kernel function.
</div>

Your answer :

# <a id="sec3"></a> 3. Application: Binary classification

We now try to do binary classification on the Mushrooms data set loaded below:

In [ ]:
data = pd.read_csv("data/mushrooms.csv")
data.head(6)

There are 2 classes, namely $y_i \in \{ e, p \}$ and many weird values for the elements of $x_i$.<br>

<div class="alert alert-warning">**Question 3.1**<br>
Use LabelEncoder from sklearn.preprocessing in order to turn the elements of $x_i$ into numerical values.
</div>

In [ ]:
# %load solutions/code5.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning">**Question 3.2**<br>
Use StandardScaler from sklearn.preprocessing in order to scale the data.
</div>

In [ ]:
# %load solutions/code6.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning">**Question 3.3**<br>
Now train an SVM classifier on the data. Try several different kernels and measure the accuracy of your predictor.<br>
**Hint:** You can split your data set into a training and testing set using train_test_split from sklearn.model_selection
</div>

In [ ]:
# %load solutions/code7.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).
